In [10]:
from os import getenv
import json
import glue_jupyter as gj
from glue_map.data import RemoteGeoData_ArcGISImageServer, Data
from glue_map.map.state import MapViewerState
from tempods.components.subset_control_widget import SubsetControlWidget

from glue.config import colormaps
from ipyleaflet import Map, Marker, LayersControl, WidgetControl, TileLayer, GeoJSON
from datetime import date, datetime, timezone, timedelta
from ipywidgets import SelectionSlider, Layout, Label, VBox, Dropdown, DatePicker, HTML, AppLayout
import pandas as pd
import numpy as np
from IPython.display import display, Javascript

from echo import delay_callback, ignore_callback
import matplotlib as mpl


In [11]:
title = HTML("<h1><font color='#FFCC33'>TEMPO Data Story Prototype</font></h1>")

In [12]:
mapapp = gj.jglue()
tempo_data = RemoteGeoData_ArcGISImageServer("https://gis.earthdata.nasa.gov/image/rest/services/C2930763263-LARC_CLOUD/",
                                            name='TEMPO')
power_data = mapapp.load_data("Power_Plants.csv")
_ = mapapp.add_data(tempo_data)

# Our remote dataset does not have real components representing latitude and longitude. We link to the only components
# it does have so that we can display this on the same viewer without trigger and IncompatibleAttribute error
mapapp.add_link(mapapp.data_collection["Power_Plants"], 'Longitude',  mapapp.data_collection["TEMPO"], 'Pixel Axis 0')
mapapp.add_link(mapapp.data_collection["Power_Plants"], 'Latitude', mapapp.data_collection["TEMPO"], 'TEMPO_NO2_L3_V03_HOURLY_TROPOSPHERIC_VERTICAL_COLUMN_BETA')

big = (power_data['Install_MW'] > 100)
med = (power_data['Install_MW'] > 10) & (power_data['Install_MW'] <= 100)
small = (power_data['Install_MW'] <= 10)
_ = power_data.add_component(big*9 + med*4 + small*1, label='Size_binned')


In [16]:
with open('coastlines.geojson', 'r') as f:
    coastdata = json.load(f)
geo_json = GeoJSON(
    data=coastdata,
    style={
        'color':'black','opacity': 1, 'fillOpacity': 0, 'weight': 0.5
    },
)


base_url="https://tiles.stadiamaps.com/tiles/stamen_toner_lines/{z}/{x}/{y}{r}.png"
labels_url="https://tiles.stadiamaps.com/tiles/stamen_toner_labels/{z}/{x}/{y}{r}.png"
api_key = getenv("STADIA_API_KEY")
if api_key is not None:
    url += f"?api_key={api_key}"
viewer_state = MapViewerState(basemap=TileLayer(url=base_url))
mapviewer = mapapp.new_data_viewer('map', data=tempo_data, state=viewer_state, show=True)
mapviewer.map.panes = {"labels": {"zIndex": 650}}

_ = mapviewer.map.add(TileLayer(url=labels_url,pane='labels'))
_ = mapviewer.map.add(geo_json)

powerplant_widget = SubsetControlWidget(power_data, mapviewer)

LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(template=Template(template='<template>\n…

In [18]:
mapviewer.figure_widget.layout = {"width": "900px", "height": "500px"}

In [21]:
timeviewer = mapapp.new_data_viewer('timeseries',data=tempo_data, show=True)

LayoutWidget(controls={'toolbar_selection_tools': BasicJupyterToolbar(template=Template(template='<template>\n…

In [20]:
timeviewer.figure.axes[1].label_offset = "-50"
timeviewer.figure.axes[1].tick_format = ".0f"
timeviewer.figure.axes[1].label = "Amount of NO2 (10^14 molecules/cm^2)"

timeviewer.figure.axes[0].label_offset = "40"
timeviewer.figure.axes[0].label = "Time (UTC)"

In [31]:
def convert_from_milliseconds(milliseconds_since_epoch):
    """Converts milliseconds since epoch to a date-time string in 'YYYY-MM-DDTHH:MM:SSZ' format."""
    dt = datetime.fromtimestamp((milliseconds_since_epoch)/ 1000, tz=timezone(offset=timedelta(hours=0), name="UTC"))
    date_time_str = dt.strftime('%H:%M')
    return date_time_str

time_values = tempo_data.get_time_steps(timeviewer.state.t_date)
time_strings = [convert_from_milliseconds(t) for t in time_values]  
time_options = [(time_strings[i], time_values[i]) for i in range(len(time_values))]

slider = SelectionSlider(description='', options=time_options,layout=Layout(width='700px', height='20px'))
dt = datetime.fromtimestamp((slider.value)/ 1000, tz=timezone(offset=timedelta(hours=0), name="UTC"))
timeviewer.timemark.x = np.array([dt, dt]).astype('datetime64[ms]')

date_chooser = DatePicker(description='Pick a Date')
date_chooser.value = date(2024, 10, 15)
def update_image(change):
    mapviewer.layers[0].state.timestep = change.new
    dt = datetime.fromtimestamp((change.new)/ 1000, tz=timezone(offset=timedelta(hours=0), name="UTC"))
    timeviewer.timemark.x = np.array([dt, dt]).astype('datetime64[ms]')

def update_date(change):
    time_values = tempo_data.get_time_steps(change.new.isoformat())
    time_strings = [convert_from_milliseconds(t) for t in time_values]  
    time_options = [(time_strings[i], time_values[i]) for i in range(len(time_values))]
    slider.options = time_options
    timeviewer.state.t_date = change.new.isoformat()
    
date_chooser.observe(update_date, 'value')

slider.observe(update_image, 'value')
control = WidgetControl(widget=slider, position='bottomleft')
_ = mapviewer.map.add(control)

# Something like this should allow us to set a min/max set of dates on the picker so the
# user does not choose a date outside of the TEMPO mission dates. But this is not working.
#date_chooser.add_class("start-date")

#script = Javascript("\
#                const query = '.start-date > input:first-of-type'; \
#                document.querySelector(query).setAttribute('min', '2023-08-01'); \
#                document.querySelector(query).setAttribute('max', '2024-10-28'); \
#        ")

In [17]:
#AppLayout(header=title,
#          left_sidebar=mapviewer.layout,
#          center=timeviewer.figure_widget,
#          right_sidebar=powerplant_widget,
#          footer=date_chooser,
#          pane_widths=[7, 3, 1],
#          pane_heights=[1, '700px', '60px'])